In [110]:
from collections import defaultdict
import numpy as np
import time, math, nltk, gensim, string
from gensim.models.keyedvectors import KeyedVectors

from sklearn.decomposition import TruncatedSVD
from nltk import word_tokenize


In [111]:
path = "glove.6B.50d.txt.w2v"
t0 = time.time()
glove = KeyedVectors.load_word2vec_format(path, binary=False)
t1 = time.time()
print("elapsed " + str(t1 - t0) + "s")

elapsed 29.252476930618286s


In [112]:
path = "glove.6B.50d.txt.w2v"
t0 = time.time()
glove = KeyedVectors.load_word2vec_format(path, binary=False)
t1 = time.time()
print("elapsed " + str(t1 - t0) + "s")

elapsed 31.28763699531555s


In [192]:
glove.wv.most_similar("table")

[('tables', 0.817743718624115),
 ('place', 0.7994711995124817),
 ('sit', 0.7571903467178345),
 ('set', 0.7330495715141296),
 ('open', 0.7316288948059082),
 ('hold', 0.730669379234314),
 ('here', 0.7287872433662415),
 ('each', 0.7207204103469849),
 ('bottom', 0.7166720032691956),
 ('top', 0.7153381109237671)]

In [8]:
svd = TruncatedSVD(n_components=2)

In [10]:
n = len(glove.vocab)
d = glove.vector_size
X_glove = np.zeros((n, d))
for i, word in enumerate(glove.vocab.keys()):
    X_glove[i,:] = glove.wv[word]
print(X_glove.nbytes)

160000000


In [11]:
t0 = time.time()
svd = TruncatedSVD(n_components=2)
svd.fit(X_glove)
t1 = time.time()
print("elapsed " + str(t1 - t0) + "s")

elapsed 2.3836846351623535s


In [49]:
def the_threshold():
    
    
    str1 = input("Enter the first sentence: ")
    
    
    vector1 = glove.wv[str1]
    
    str2 = input("Enter the second sentence: ")
    vector2 = glove.wv[str2]
    diff = vector1 - vector2

    thresh = math.sqrt(abs(np.sum(diff))*21)
    
    
    return thresh

In [50]:
the_threshold()

Enter the first word: cheap
Enter the second word: discount


10.023405840638635

In [63]:
def the_sent_threshold():
    
    str1 = input("Enter the first sentence: ")
    
    list1 = nltk.word_tokenize(str1)
    
    vector1list = []
    
    for word in list1:
        
        vector1list.append(glove.wv[word])
        
        
    vector1_array = np.array(vector1list)
    
    str2 = input("Enter the second sentence: ")
    
    list2 = nltk.word_tokenize(str2)
    
    vector2list = []
    
    for word in list2:
        
        vector2list.append(glove.wv[word])
        
        
    vector2_array = np.array(vector2list)
    
    diff = vector1_array - vector2_array
    
    
    
    thresh = math.sqrt(abs(np.sum(diff))*21)
    
    
    return thresh

In [64]:
the_sent_threshold()

Enter the first sentence: i am hungry
Enter the second sentence: bob is hungry


16.609371267840448

In [65]:
the_sent_threshold()

Enter the first sentence: i am tired
Enter the second sentence: i am hungry


8.228929445412762

In [66]:
the_sent_threshold()

Enter the first sentence: i am tired
Enter the second sentence: i am sleepy


7.445069668343312

In [70]:
the_sent_threshold()

Enter the first sentence: my router is not working properly
Enter the second sentence: i want to purchase a plan


22.483849005680266

In [105]:
the_sent_threshold()

Enter the first sentence: my router is not working
Enter the second sentence: my router is not working properly


ValueError: operands could not be broadcast together with shapes (5,50) (6,50) 

In [186]:
the_sent_threshold()

Enter the first sentence: i am moving to a new apartment but want to keep my services
Enter the second sentence: can i simply move my router to the new apartment and self install


2.765518445047413

In [95]:
def tokenize(text, preserve_case=True, filter_text=True, stopwords=None, punc=string.punctuation):
    if filter_text and stopwords is None:
        with open("stopwords.txt", 'r') as r:
            stops = []
            for line in r:
                stops += [i.strip() for i in line.split('\t')]
    stopwords = stops
    tokens = word_tokenize(text)
    indices = []
    if filter_text:
        for i in range(len(tokens)):
            word = tokens[i]
            # print(word, word in punc, word.lower() in stopwords)
            if word in punc or word.lower() in stopwords:
                indices.append(i)
        tokens = list(np.delete(np.array(tokens), np.array(indices)))
    if preserve_case:
        return tokens
    return [i.lower() for i in a]

In [187]:
def sent_threshold1():
    
    str1 = input("Enter the first sentence: ")
    
    list1 = tokenize(str1)
    
    vector1list = []
    
    for word in list1:
        
        vector1list.append(glove.wv[word])
        
        
    vector1_array = np.array(vector1list)
    
    str2 = input("Enter the second sentence: ")
    
    list2 = tokenize(str2)
    
    vector2list = []
    
    for word in list2:
        
        vector2list.append(glove.wv[word])
        
        
    vector2_array = np.array(vector2list)
    
    
    dim1 = vector1_array.shape[0]
    dim2 = vector2_array.shape[0]
    
    if dim1 > dim2 :
        
        diff = dim1 - dim2
        
        for i in range(diff):
            arr = np.mean(vector2_array,axis=0)
            
            vector2_array = np.vstack([vector2_array,arr])
            
    if dim2 > dim1 :
        
        diff = dim2 - dim1
        
        for i in range(diff):
            arr = np.mean(vector1_array,axis=0)
            
            vector1_array = np.vstack([vector1_array,arr])
            
    
    diff = vector1_array - vector2_array
    
    
    
    thresh = math.sqrt(abs(np.sum(diff))*21)
    
    
    return thresh

In [184]:
sent_threshold1()

Enter the first sentence: i am moving to a new apartment but want to keep my services
Enter the second sentence: can I simply move my router to the new apartment and self install?


15.360468609660591

In [188]:
sent_threshold1()

Enter the first sentence: i am moving to a new apartment but want to keep my services
Enter the second sentence: can I simply move my router to the new apartment and self install?


9.612487288963932

In [190]:
sent_threshold1()

Enter the first sentence: i called and i forgot my password
Enter the second sentence: she said to go on internet and create a new password and i am having problems


12.081642181614445

In [191]:
sent_threshold1()

Enter the first sentence: why is anything i watch very pixelated and every video buffer even when only one device is using the wifi
Enter the second sentence: i called company and i forgot my password


16.905288640750847

ROUTER PROBLEMS
